In [ ]:
import numpy as np

In [ ]:
m = 3
m2 = m ** 2
q = np.zeros(m2)
q[m2 // 2] = 1

In [ ]:
q

In [ ]:
def get_P(m, p_up, p_down, p_left, p_right):
    m2 = m ** 2
    P = np.zeros((m2, m2))
    ix_map = {i + 1: (i // m, i % m) for i in range(m2)}
    for i in range(m2):
        for j in range(m2):
            r1, c1 = ix_map[i + 1]
            r2, c2 = ix_map[j + 1]
            rdiff = r1 - r2
            cdiff = c1 - c2
            if rdiff == 0:
                if cdiff == 1:
                    P[i, j] = p_left
                elif cdiff == -1:
                    P[i, j] = p_right
                elif cdiff == 0:
                    if r1 == 0:
                        P[i, j] += p_down
                    elif r1 == m - 1:
                        P[i, j] += p_up
                    if c1 == 0:
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
            elif rdiff == 1:
                if cdiff == 0:
                    P[i, j] = p_down
            elif rdiff == -1:
                if cdiff == 0:
                    P[i, j] = p_up
    return P

In [ ]:
P = get_P(3, 0.2, 0.3, 0.25, 0.25)

In [ ]:
np.matmul(q, P)

In [ ]:
n = 1
Pn = np.linalg.matrix_power(P, n)
np.matmul(q, Pn)

In [ ]:
n = 3
Pn = np.linalg.matrix_power(P, n)
np.round(np.matmul(q, Pn), 3)

In [ ]:
n = 10
Pn = np.linalg.matrix_power(P, n)
np.round(np.matmul(q, Pn), 3)

In [ ]:
n = 100
Pn = np.linalg.matrix_power(P, n)
np.round(np.matmul(q, Pn), 3)

### Ergodic MC

In [ ]:
from scipy.stats import itemfreq

In [ ]:
s = 4
n = 10 ** 6
visited = [s]
for t in range(n):
    s = np.random.choice(m2, p=P[s, :])
    visited.append(s)

In [ ]:
itemfreq(visited)

In [ ]:
P

# MRP

## Modify P

In [ ]:
P = np.zeros((m2 + 1, m2 + 1))
P[:m2, :m2] = get_P(3, 0.2, 0.3, 0.25, 0.25)
for i in range(m2):
    P[i, m2] = P[i, i]
    P[i, i] = 0
P[m2, m2] = 1

In [ ]:
P

In [ ]:
n = 10 ** 5
avg_rewards = np.zeros(m2)
for s in range(9):
    for i in range(n):
        crashed = False
        s_next = s
        episode_reward = 0
        while not crashed:
            s_next = np.random.choice(m2 + 1, p=P[s_next, :])
            if s_next < m2:
                episode_reward += 1
            else:
                crashed = True
        avg_rewards[s] += episode_reward
avg_rewards /= n

In [ ]:
np.round(avg_rewards, 2)

In [ ]:
(1 + 2.45) * 0.25 + (1 + 2.44) * 0.25 + 0.2 * (1+2.81) + 0.3*(1+2.12)

### Analytically calculate the state values

In [ ]:
R = np.ones(m2 + 1)
R[-1] = 0
inv = np.linalg.inv(np.eye(m2 + 1) - 0.9999 * P)
v = np.matmul(inv, np.matmul(P, R))
print(np.round(v, 2))

### Estimating State Values

In [ ]:
def estimate_state_values(P, m2, threshold):
    v = np.zeros(m2 + 1)
    max_change = threshold
    terminal_state = m2 
    while max_change >= threshold:
        max_change = 0
        for s in range(m2 + 1):
            v_new = 0
            for s_next in range(m2 + 1):
                r = 1 * (s_next != terminal_state)
                v_new += P[s, s_next] * (r + v[s_next])
            max_change = max(max_change, np.abs(v[s] - v_new))
            v[s] = v_new
    return np.round(v, 2)

In [ ]:
estimate_state_values(P, m2, 0.005)